In [ ]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp

openjdk version "1.8.0_275"
OpenJDK Runtime Environment (build 1.8.0_275-8u275-b01-0ubuntu1~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.275-b01, mixed mode)
     |████████████████████████████████| 215.7MB 57kB/s 
     |████████████████████████████████| 204kB 45.1MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130389 sha256=3e9961149ab2a6cb8c5aaaa884a96536c3f0f80203adf6b55f1d06f74d7de1ae
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark
     |████████████████████████████████| 133kB 4.2MB/s 


In [ ]:
import sparknlp
spark = sparknlp.start()

print("Spark NLP version")
sparknlp.version()
print("Apache Spark version")
spark.version

Spark NLP version
Apache Spark version


'2.4.4'

In [ ]:
from sparknlp.pretrained import PretrainedPipeline 

In [ ]:
import pandas as pd


In [45]:
lines = pd.read_csv("/content/drive/My Drive/archive-3/covid19_tweets.csv")
lines.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False


In [ ]:
lines = lines['text'].values.tolist()
lines

In [48]:
pipeline = PretrainedPipeline('recognize_entities_dl', 'en')

recognize_entities_dl download started this may take some time.


KeyboardInterrupt: ignored

In [ ]:
lines = lines[:1000]
entities_dic = {}
for line in lines:
  result = pipeline.annotate(line)
  for e in result['entities']:
    count = entities_dic.get(e,0)
    entities_dic[e] = count+1
dic2 = sorted(entities_dic,reverse=True)

In [ ]:
result['entities']

['@HRDMinistry @DrRPNishank.Please']

In [ ]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|                 ner|            entities|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|If I smelled the ...|[[document, 0, 13...|[[document, 0, 13...|[[token, 0, 1, If...|[[word_embeddings...|[[named_entity, 0...|                  []|
|Hey @Yankees @Yan...|[[document, 0, 13...|[[document, 0, 13...|[[token, 0, 2, He...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 22, H...|
|@diane3443 @wdunl...|[[document, 0, 13...|[[document, 0, 81...|[[token, 0, 9, @d...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 20, 41, ...|
|@brookbanktv The ...|[[document, 0, 13...|[[document, 0, 13...|[[token, 0, 11, @...|[[word_embeddings...|

In [ ]:
entities = result.select('entities.result')

In [ ]:
entities.show()

KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.tokenize import TweetTokenizer
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.sentiment.util import mark_negation
from gensim.models import doc2vec
from nltk.corpus import sentiwordnet as swn

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [59]:
sw = stopwords.words('english')
word_tokenizer = TweetTokenizer(strip_handles=True, preserve_case=False, reduce_len=True)
def remove_punc_and_stopword_2(text):
    punc_removed = [word for word in text if word not in string.punctuation]
    punc_removed = ''.join(punc_removed)
    punc_removed = word_tokenizer.tokenize(punc_removed)
    # vocabularies = []
    for token in punc_removed :
        if token.lower() not in sw:
            # vocabularies.append(token.lower())
    return vocabularies

NameError: ignored

In [50]:
import pyspark
from pyspark import SparkContext
from pyspark import SparkConf
conf=SparkConf().setAppName("miniProject").setMaster("local[*]")
sc=SparkContext.getOrCreate(conf)

In [51]:
data = sc.parallelize(lines)

In [52]:
data.take(10)

['If I smelled the scent of hand sanitizers today on someone in the past, I would think they were so intoxicated that… https://t.co/QZvYbrOgb0',
 "Hey @Yankees @YankeesPR and @MLB - wouldn't it have made more sense to have the players pay their respects to the A… https://t.co/1QvW0zgyPu",
 '@diane3443 @wdunlap @realDonaldTrump Trump never once claimed #COVID19 was a hoax. We all claim that this effort to… https://t.co/Jkk8vHWHb3',
 '@brookbanktv The one gift #COVID19 has give me is an appreciation for the simple things that were always around me… https://t.co/Z0pOAlFXcW',
 '25 July : Media Bulletin on Novel #CoronaVirusUpdates #COVID19 \n@kansalrohit69 @DrSyedSehrish @airnewsalerts @ANI… https://t.co/MN0EEcsJHh',
 "#coronavirus #covid19 deaths continue to rise. It's almost  as bad as it ever was.  Politicians and businesses want… https://t.co/hXMHooXX2C",
 'How #COVID19 Will Change Work in General (and recruiting, specifically) via/ @ProactiveTalent #Recruiting… https://t.co/bjZxzGPMbK

In [56]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [61]:
vab = data.flatMap(remove_punc_and_stopword_2)
vab.take(3)

['smelled', 'scent', 'hand']

In [66]:
vab_count = vab.map(lambda x:(x,1)).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1] > 10).sortBy(lambda x:x[1], False)
vab_count.take(10)

[('…', 138649),
 ('covid', 117973),
 ('19', 113100),
 ('httpstco', 28918),
 ('’', 19880),
 ('cases', 19004),
 ('coronavirus', 15196),
 ('new', 14988),
 ('amp', 12302),
 ('people', 9403)]